In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from dhdrnet.util import DATA_DIR, ROOT_DIR


figdir = ROOT_DIR / "figures"

In [107]:
df = (
    pd.read_csv(
        ROOT_DIR / "precomputed_data" / "store_2020-10-27.csv", index_col=0
    )
    .drop(columns="ev1")
    .rename(columns={"ev2": "ev"})
)
names = pd.read_csv(ROOT_DIR / "precomputed_data" / "test_current.csv")
df = df[df["name"].isin(names["names"])]
df = df.drop_duplicates()
df

,ev,name,metric,score
1584,-3.00,5066_20160724_121618_110,rmse,16.201134
1585,-3.00,5066_20160724_121618_110,psnr,23.939895
1586,-3.00,5066_20160724_121618_110,ssim,0.965922
1587,-3.00,5066_20160724_121618_110,perceptual,0.047302
1588,-2.75,5066_20160724_121618_110,rmse,16.540948
...,...,...,...,...
523723,5.75,0043_20160923_122514_781,perceptual,0.148934
523724,6.00,0043_20160923_122514_781,rmse,35.136912
523725,6.00,0043_20160923_122514_781,psnr,17.215532
523726,6.00,0043_20160923_122514_781,ssim,0.877194


In [76]:
grouped = df.set_index("name").groupby(["metric"])
grouped.describe()["score"][["mean", "std", "min", "max"]]

,mean,std,min,max
metric,,,,
perceptual,0.152325,0.112680,0.001084,0.640015
psnr,18.004786,5.801761,7.166248,50.651614
rmse,39.081830,23.139358,0.748100,111.744920
ssim,0.769130,0.231517,0.045940,0.998201


In [77]:
idxmins = (
    df[(df["metric"] == "perceptual") | (df["metric"] == "rmse")]
    .groupby(["name", "metric"])["score"]
    .idxmin()
)
best_scores_min = df.loc[idxmins]
best_scores_min
# this is the minimum score for each metric for each image
# unfortunately we want min for mse and perceptual
# and maxes for psnr, ssim

,name,ev,metric,score
5432911,0006_20160721_170707_736,2.00,perceptual,0.031581
5432936,0006_20160721_170707_736,3.75,rmse,11.903806
8294487,0006_20160721_175239_909,2.50,perceptual,0.058179
8294508,0006_20160721_175239_909,4.00,rmse,9.563386
10907243,0006_20160721_181525_062,3.75,perceptual,0.049348
...,...,...,...,...
11840348,c483_20150901_195108_436,3.00,rmse,19.015955
1679695,c483_20150901_214630_530,2.00,perceptual,0.059056
1679716,c483_20150901_214630_530,3.50,rmse,8.471777
1721199,c483_20150901_221141_886,4.00,perceptual,0.043705


In [78]:
idxmaxs = (
    df[(df["metric"] == "psnr") | (df["metric"] == "ssim")]
    .groupby(["name", "metric"])["score"]
    .idxmax()
)
best_scores_max = df.loc[idxmaxs]
best_scores_max

,name,ev,metric,score
5432937,0006_20160721_170707_736,3.75,psnr,26.617087
5432910,0006_20160721_170707_736,2.00,ssim,0.975438
8294509,0006_20160721_175239_909,4.00,psnr,28.518569
8294510,0006_20160721_175239_909,4.00,ssim,0.965298
10907245,0006_20160721_181525_062,4.00,psnr,26.302924
...,...,...,...,...
11840350,c483_20150901_195108_436,3.00,ssim,0.971811
1679717,c483_20150901_214630_530,3.50,psnr,29.571314
1679722,c483_20150901_214630_530,3.75,ssim,0.970954
1721189,c483_20150901_221141_886,3.50,psnr,31.740585


In [79]:
best_scores_min.groupby("metric").describe()["score"][["mean", "std"]]

,mean,std
metric,,
perceptual,0.037905,0.024077
rmse,10.717247,5.932719


In [80]:
best_scores_max.groupby("metric").describe()["score"][["mean", "std"]]

,mean,std
metric,,
psnr,29.075697,5.774687
ssim,0.973383,0.014968


In [81]:
df.loc[df["ev"] == 3.5].groupby("metric").describe()["score"][["mean", "std"]]

,mean,std
metric,,
perceptual,0.055462,0.038827
psnr,26.724160,5.663801
rmse,14.507083,10.176345
ssim,0.961232,0.023200


In [82]:
df.loc[df["ev"] == 3.5].groupby("metric").describe()["score"][["mean", "std"]].to_latex(
    figdir / "fixed_choice.tex"
)

In [84]:
cdf = pd.read_csv(ROOT_DIR / "precomputed_data" / "store_updown_2020-10-26.csv").drop_duplicates()
cdf.loc[cdf["metric"] == "rmse", "score"] = cdf.loc[
    cdf["metric"] == "rmse", "score"
].apply(np.sqrt)

cdf

,name,metric,ev,score
0,0030_20151008_105249_589,rmse,1,39.161667
1,0030_20151008_105249_589,psnr,1,16.273580
2,0030_20151008_105249_589,ssim,1,0.812035
3,0030_20151008_105249_589,rmse,2,25.583744
4,0030_20151008_105249_589,psnr,2,19.971522
...,...,...,...,...
10855,0043_20160923_122514_781,psnr,4,19.521871
10856,0043_20160923_122514_781,ssim,4,0.928805
10857,0043_20160923_122514_781,rmse,5,30.929737
10858,0043_20160923_122514_781,psnr,5,18.323279


In [85]:
cidxmins = (
    cdf[(cdf["metric"] == "perceptual") | (cdf["metric"] == "rmse")]
    .groupby(["name", "metric"])["score"]
    .idxmin()
)
cbest_scores_min = cdf.loc[cidxmins]
cbest_scores_min
# this is the minimum score for each metric for each image
# unfortunately we want min for mse and perceptual
# and maxes for psnr, ssim

,name,metric,ev,score
3933,0006_20160721_170707_736,rmse,2,5.948025
6009,0006_20160721_175239_909,rmse,4,9.848779
7899,0006_20160721_181525_062,rmse,4,9.309660
2436,0006_20160722_100126_028,rmse,3,8.724895
5181,0006_20160722_100951_201,rmse,3,6.278645
...,...,...,...,...
6138,c483_20150901_173210_443,rmse,2,8.954333
9219,c483_20150901_194029_165,rmse,4,9.003112
8589,c483_20150901_195108_436,rmse,4,12.383640
1221,c483_20150901_214630_530,rmse,3,11.204544


In [86]:
cidxmaxs = (
    cdf[(cdf["metric"] == "psnr") | (cdf["metric"] == "ssim")]
    .groupby(["name", "metric"])["score"]
    .idxmax()
)
cbest_scores_max = cdf.loc[cidxmaxs]
cbest_scores_max

,name,metric,ev,score
3934,0006_20160721_170707_736,psnr,2,32.643347
3935,0006_20160721_170707_736,ssim,2,0.993332
6010,0006_20160721_175239_909,psnr,4,28.263156
6011,0006_20160721_175239_909,ssim,4,0.967660
7900,0006_20160721_181525_062,psnr,4,28.752127
...,...,...,...,...
8591,c483_20150901_195108_436,ssim,4,0.976068
1222,c483_20150901_214630_530,psnr,3,27.142920
1223,c483_20150901_214630_530,ssim,3,0.960959
1255,c483_20150901_221141_886,psnr,4,29.356700


In [87]:
cmin = cbest_scores_min.groupby("metric").describe()["score"][["mean", "std"]]
cmax = cbest_scores_max.groupby("metric").describe()["score"][["mean", "std"]]
pd.concat([cmin, cmax])

,mean,std
metric,,
rmse,10.060834,2.880453
psnr,28.448983,2.632520
ssim,0.976898,0.012301


In [88]:
cdf.groupby(["ev", "metric"]).describe()["score"][["mean", "std"]].to_latex(
    figdir / "conv_choice.tex"
)
cdf.groupby(["ev", "metric"]).describe()["score"][["mean", "std"]]

mean        std
ev metric                      
1  psnr    16.144820   4.237501
   rmse    44.319712  19.385214
   ssim     0.707827   0.226675
2  psnr    20.877057   5.383900
   rmse    27.334192  14.555937
   ssim     0.848401   0.141950
3  psnr    25.248919   3.031392
   rmse    14.784704   5.051738
   ssim     0.943758   0.033221
4  psnr    26.384000   3.998241
   rmse    13.644772   7.042175
   ssim     0.967228   0.020001
5  psnr    19.700461   3.322998
   rmse    28.345435  10.790692
   ssim     0.880723   0.058906

In [161]:
def parse_preds(pred_str):
    splits = pred_str.split(" ")
    return [float(s) for s in splits[1:-1] if len(s) > 0]

model_df = pd.read_csv(ROOT_DIR/"precomputed_data" / "mobile_preds.csv", index_col=0)
model_df["pred"] = model_df["pred"].apply(parse_preds)
model_df

,name,pred
0,0006_20160721_170707_736,"[3.25, 3.5, 2.75, 2.5, 3.75, 2.25, 2.0, 4.0]"
1,0006_20160721_175239_909,"[3.75, 3.25, 4.0, 3.0, 4.25, 2.75, 4.5, 2.5]"
2,0006_20160721_181525_062,"[3.75, 4.25, 3.5, 3.25, 4.5, 4.75, 3.0, 2.75, ..."
3,0006_20160722_100126_028,"[3.5, 3.0, 2.75, 3.75, 2.5, 2.25, 4.0, 4.25, 2.0]"
4,0006_20160722_100951_201,"[-0.75, 4.25, -0.25, 4.5, 3.75, 4.0, 5.0, 3.5,..."
...,...,...
719,c483_20150901_173210_443,"[4.5, -0.5, 4.25, 3.75, 4.0, 5.0, 5.5, -1.0]"
720,c483_20150901_194029_165,"[4.0, 3.75, 4.5, 3.5, 3.25, 4.75, 5.0, 5.5]"
721,c483_20150901_195108_436,"[4.5, 5.0, 3.25, 4.75, 2.75, 3.5, 5.25, 4.25]"
722,c483_20150901_214630_530,"[3.0, 3.5, 2.75, 2.5, 3.75, 2.25, 4.0, 2.0]"


In [162]:
from dhdrnet.util import get_scores_for_preds, get_topk_score_df

In [163]:
# joined = df.set_index("name").join(model_df.set_index("name"))
# joined.iloc[0]
df.iloc[0]

ev                              -3
name      5066_20160724_121618_110
metric                        rmse
score                      16.2011
Name: 1584, dtype: object

In [164]:
model_df.iloc[0]

name                        0006_20160721_170707_736
pred    [3.25, 3.5, 2.75, 2.5, 3.75, 2.25, 2.0, 4.0]
Name: 0, dtype: object

In [171]:
t = get_scores_for_preds(model_df, df)
t.groupby("metric").describe()["score"]  # These are the top-10 accuracies

,count,mean,std,min,25%,50%,75%,max
metric,,,,,,,,
perceptual,6300.0,0.069632,0.056401,0.001084,0.039397,0.061507,0.087838,0.639502
psnr,6300.0,24.072651,4.728202,9.906567,20.989723,23.976195,26.844718,50.651614
rmse,6300.0,18.290789,9.721521,0.748100,11.595895,16.133568,22.753804,81.510180
ssim,6300.0,0.929619,0.095888,0.045940,0.925006,0.954945,0.968363,0.998201


In [175]:
r = get_topk_score_df(t,k=1)
r

,ev,name,metric,score
188143,2.00,0006_20160721_170707_736,perceptual,0.031581
188168,3.75,0006_20160721_170707_736,rmse,11.903806
286935,2.50,0006_20160721_175239_909,perceptual,0.058179
286956,4.00,0006_20160721_175239_909,rmse,9.563386
377243,3.75,0006_20160721_181525_062,perceptual,0.049348
...,...,...,...,...
413378,3.25,c483_20150901_195108_436,ssim,0.970424
71381,3.50,c483_20150901_214630_530,psnr,29.571314
71386,3.75,c483_20150901_214630_530,ssim,0.970954
71961,3.75,c483_20150901_221141_886,psnr,30.966746


In [176]:
r.groupby("metric").describe()

ev                                                  score  \
            count      mean       std  min   25%  50%   75%   max  count   
metric                                                                     
perceptual  724.0  2.695442  1.790108 -3.0  2.25  3.5  3.75  4.50  724.0   
psnr        724.0  3.078729  1.625198 -3.0  3.25  3.5  3.75  5.50  724.0   
rmse        724.0  3.078729  1.625198 -3.0  3.25  3.5  3.75  5.50  724.0   
ssim        724.0  3.022790  1.463892 -3.0  2.50  3.5  3.75  5.25  724.0   

                                                                             \
                 mean       std        min        25%        50%        75%   
metric                                                                        
perceptual   0.039571  0.025813   0.001084   0.020349   0.037815   0.053381   
psnr        28.876916  5.503402  17.098601  25.510544  28.475609  31.247695   
rmse        10.862788  6.015123   0.748100   6.984827   9.610804  13.521135   
ssim         0.971703  0.017268   0.875470   0.964549   0.973921   0.983126   

                       
                  max  
metric                 
perceptual   0.145628  
psnr        50.651614  
rmse        35.613130  
ssim         0.998201